# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [9]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

print(adult_dt.head())


   age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital-status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  income  
0          2174             0              40   United-States       

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [11]:
from sklearn.model_selection import train_test_split
# Create the feature matrix X (excluding the 'income' column)
X = adult_dt.drop(columns='income')

# Create the target vector Y (only the 'income' column)
Y = adult_dt['income']

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Display the shapes of the training and testing sets
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'Y_train shape: {Y_train.shape}')
print(f'Y_test shape: {Y_test.shape}')

X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*
The random state in the 'train_test_split' function is used to ensure that the splitting of the data into training and testing sets is reproducibles. When I set 'random_state' to a specific integer, it ensures that the split between the training and testing sets will be the same when the code is runned.

It ensure everyone who run the code could vertify and validate findings y eunsring that the results can be consistently reproduced under the same conditions.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [29]:
import numpy as np
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Identify numerical and categorical columns
num_cols = X.select_dtypes(include=np.number).columns.tolist()
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
            ('scaler', RobustScaler())
        ]), num_cols),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
        ]), cat_cols)
    ]
)

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
# Create the model pipeline
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [31]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score, roc_auc_score, log_loss

#Define the scoring metrics
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'balanced_accuracy': make_scorer(balanced_accuracy_score),
    'roc_auc': make_scorer(roc_auc_score),
    'log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True)
}

#Perform cross-validation
cv_results = cross_validate(pipeline, X_train, Y_train, cv=5, scoring=scoring, return_train_score=True )

#Report the results
print("Cross-Validation Results:")
for key in scoring.keys():
    print(f"{key.capitalize()} - Train: {np.mean(cv_results[f'train_{key}']):.4f}, Validation: {np.mean(cv_results[f'test_{key}']):.4f}")

c:\Users\jolin\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\jolin\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Cross-Validation Results:
Accuracy - Train: 1.0000, Validation: 0.8542
Balanced_accuracy - Train: 1.0000, Validation: 0.7763
Roc_auc - Train: 1.0000, Validation: 0.7763
Log_loss - Train: -0.0816, Validation: -0.3680


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [32]:
fold_results = pd.DataFrame({
    'Accuracy(Train)': cv_results['train_accuracy'],
    'Accuracy(Validation)': cv_results['test_accuracy'],
    'Balanced Accuracy(Train)': cv_results['train_balanced_accuracy'],
    'Balanced Accuracy(Validation)': cv_results['test_balanced_accuracy'],
    'ROC AUC(Train)': cv_results['train_roc_auc'],
    'ROC AUC(Validation)': cv_results['test_roc_auc'],
    'Negative Log Loss(Train)': -cv_results['train_log_loss'],
    'Negative Log Loss(Validation)': -cv_results['test_log_loss']
})

#Sort by Negaitve Log Loss(Validation)
fold_results_sorted = fold_results.sort_values(by='Negative Log Loss(Validation)')

#Display sorted fold-level results
print('Fold-Level Results Sorted by Negative Log Loss(Validation):')
print(fold_results_sorted)

Fold-Level Results Sorted by Negative Log Loss(Validation):
   Accuracy(Train)  Accuracy(Validation)  Balanced Accuracy(Train)  \
3         1.000000              0.859807                  1.000000   
0         0.999945              0.850625                  0.999887   
1         1.000000              0.850406                  1.000000   
2         0.999945              0.854103                  0.999887   
4         1.000000              0.856077                  1.000000   

   Balanced Accuracy(Validation)  ROC AUC(Train)  ROC AUC(Validation)  \
3                       0.782859        1.000000             0.782859   
0                       0.774484        0.999887             0.774484   
1                       0.771881        1.000000             0.771881   
2                       0.776017        0.999887             0.776017   
4                       0.776089        1.000000             0.776089   

   Negative Log Loss(Train)  Negative Log Loss(Validation)  
3                  

Calculate the mean of each metric. 

In [34]:
mean_metrics = fold_results.mean()
print('\nMean Metrics:')
print(mean_metrics)


Mean Metrics:
Accuracy(Train)                  0.999978
Accuracy(Validation)             0.854204
Balanced Accuracy(Train)         0.999955
Balanced Accuracy(Validation)    0.776266
ROC AUC(Train)                   0.999955
ROC AUC(Validation)              0.776266
Negative Log Loss(Train)         0.081610
Negative Log Loss(Validation)    0.368014
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [39]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score

# Fit pipeline on training data
pipeline.fit(X_train, Y_train)

# Predict probabilities for X_test
Y_pred_proba = pipeline.predict_proba(X_test)

# Calculate ROC AUC using prediction probabilities
roc_auc = roc_auc_score(Y_test, Y_pred_proba[:, 1])

# Calculate negative log loss using prediction probabiliites
neg_log_loss = log_loss(Y_test, Y_pred_proba)

# Calculate accuracy
accuracy = accuracy_score(Y_test, pipeline.predict(X_test))

# Calculate balanced accuracy
balanced_accuracy = balanced_accuracy_score(Y_test, pipeline.predict(X_test))

# Display results as a dictionary
test_results = {
    'ROC AUC (Test)': roc_auc,
    'Negative Log Loss (Test)': neg_log_loss,
    'Accuracy(Test)': accuracy,
    'Balanced Accuracy(Test)': balanced_accuracy
}

print("\nTest Set Performance Metrics:")
print(test_results)


Test Set Performance Metrics:
{'ROC AUC (Test)': 0.8999828704291436, 'Negative Log Loss (Test)': 0.39726506589398913, 'Accuracy(Test)': 0.8551540587572934, 'Balanced Accuracy(Test)': 0.7751631656838177}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)
1. SSimplifies Model Training
2. Ensures Consistency
3. Improves Readability and Maintainability

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.